# Bike Maintenance and Repair Analysis
Working as Head Mechanic and Assistant Area Manager for 2 years for the #1 Recreation Rentals by Entrepreneur Magazine for 5 years. I set up a free product management system for the four locations I grew into with 10 product types with 300+ total products with 400 reports from January 2020 to September 2021 where I and a part time mechanic managed with no prior mechanic experience using Notion for managment and data collection and Chillipepper API for a QR Code for team members to report when bikes were down to give live status on locations.

## Objective of Analysis
- Analyze the frequency and types of bike breakdowns
- Evaluate the efficiency of the repair process
- Identify trends and patterns in bike maintenance

## Dataset Description
- Down Date : The date and time the bike was reported broken
- Fixed Date : The date and time the bike was reported fixed
- Location : Catagorical location of the bike that was broken 
    - Mile Square Park ( Edinger and Warner) Waterfront Adventures, Yorba Linda Regional Park
- Description: Description by team member of the problem of the product
- Product: Catagorical of 10 products including; 
    Swan Boats, Kayaks, Single and Double Surreys, Specialty Products (Deuce Coupe, Quad Sport, Chopper), Cruisers (Adult/Kids), E-Bikes (Waterfront Only),and Tandem Bikes.
    
## Analysis Questions
1. How much productivity did the system bring over the two years?
   - With a moving average per week beginning in November in 2020 with around 150 days to having a moving average to be withing 50 to 0 days by March 2021 and keeping it below 25 days during peak months in 2021. See Productivity Heading 
2. What was busiest location in relation to the parks attendence? (Only Mile Square)
3. Rank the most successfully maintained locaitons? Does the priorty match?
    Under Medians and Averages by Locations
    1. Yorba (Average of 20 days and Median of 6 days) 
    2. Edinger (Average of 21 days and Median of 7 days)
    3. Watefront (Average of 39 days and Median of 14 days)
    4. Warner (Average of 45 days and median of 42 days) 
4. Was there any reoccuring problems? Flat Tires, broken wheels, etc.?
5. Did implementing a daily to do list and self inspection for mechanics and team members for slow times have a positive effect on the product? 
    - (08/20) Daily Team Memeber Task Sheet
        - Including bike maintaince tasks like putting air in the tires, and tightening chains.
    - (03/21) Mechanic Self Inspection Form 

# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import warnings
import datetime
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import preprocessing 


In [ ]:
df = pd.read_csv('/kaggle/input/wheelfun-database/MB_Reports.csv')

# Uncleaned Data Exploring


In [ ]:
plt.figure(figsize=(20,6))
df['Location'].hist()
plt.title('Reports by Location')

In [ ]:
plt.figure(figsize=(20,6))
df['Product'].hist()
plt.title('Reports by Product')

In [ ]:
df.count()
# Should be about 450 - 540 total reports based on required input (Description, Location, and Status column counts)
# Deleting the Location_Part columns (Ex.'Warner Part') due to operational use only for orederiing parts.
# Deleting the Attatchment feature column that stored photos of the broken part that needed to replaced for easy communication

In [ ]:
df.head()

In [ ]:
df.tail()
# Lots of null values when first starting to use the reports that later was implemented

In [ ]:
print("Data Types")
df.dtypes
# Tackle data types for all columns

In [ ]:
print("Null Values")
df.isnull().sum()
# I have a half empty file

In [ ]:
df['Status'].value_counts()

# Data Cleaning
- [X] Cleaning the Columns
- [X] Fill null values
- [X] Change Data Types
- [X] Clean the Status Column
- [X] Clean the Category Column 
- [ ] Clean Product_Num Column


## Cleaning the Column Name

In [ ]:
df.columns

In [ ]:
#Renaming columns for better use cases ('Down Date' to 'Down_Date')
df = df.rename(columns = {"Down Date":"Down_Date",'Product #':'Product_Num','Last Edited':'Fixed_Date','Mechanic Notes':'Mechanic_Notes','1 st attempt date':'1stdate', 'Edinger Part':'EdingerPart', 'Warner Part':'Warnerpart', 'Waterfront Part':'WaterfrontPart',
       'Yorba Part':'YorbaPart','Part Comment':'PartComment','Attatchment feauture':'Pictures','Last Edited by':'Last_edited_by'})  
# Dropping unwanted columns 
df.drop(['Pictures','Last_edited_by','Reporter','EdingerPart', 'Warnerpart', 'WaterfrontPart', 'YorbaPart'],axis=1, inplace=True)
# Sort values by Down_Date
df = df.sort_values('Down_Date')
# Capatalizing the first letter of the Description column
df['Description'] = df['Description'].str.capitalize()
df.describe()

In [ ]:
df.columns

### Fill nulls 

In [ ]:
# Dropping Rows with null values based on no information in Product or Location column
df = df.dropna(subset=['Location','Product'])
df.isnull().sum()

###  Status Column
- [X] Label Unfinsihed reports as Not_complete

In [ ]:
df['Status'].isnull().sum()

In [ ]:
# Replace null values in the 'Status' column with 'Not_Complete'
df['Status'].fillna('Not_Complete', inplace=True)
# Plot Not_Complete 
not_complete = df[df['Status']=='Not_Complete']
df['Status'].value_counts()

### Change Data types

In [ ]:
# convert 'Location', 'Categorical', 'Product', and 'Status' columns to categorical data type
df['Location'] = df['Location'].astype('category')
df['Category'] = df['Category'].astype('category')
df['Product'] = df['Product'].astype('category')
df['Status'] = df['Status'].astype('category')

# convert 'Down_Date', 'Fixed_Date', and '1stdate' columns to pandas datetime data type
df['Down_Date'] = pd.to_datetime(df['Down_Date'])
df['Fixed_Date'] = pd.to_datetime(df['Fixed_Date'])

# convert 'description' and 'Mechanic_Notes' columns to string data type
df['Description'] = df['Description'].astype(str)
df['Mechanic_Notes'] = df['Mechanic_Notes'].astype(str)

# convert 'mechanic' column to categorical data type
df['Mechanic'] = df['Mechanic'].astype('category')

# convert 'PartComment' column to string data type
df['PartComment'] = df['PartComment'].astype(str)
print('Cleaned Data Types')
df.dtypes

### Category Report Column
- [X] Replace Mechanics Attention and Mechanic to Down
- [X] Fill Null values with Down
- [X] Drop maintainence routines and Cant Rent products


In [ ]:
# replace 'Mechanics attention' to 'Down'
df = df.replace('Mechanics Attention', 'Down')

# Replace Mechanic to Down
df= df.replace('Mechanic','Down')

# Fill null values with 'Down'
df['Category'] = df['Category'].fillna('Down')

# Deleting the ones that Can't be rented which would be an outliers in the dataset
df = df[df['Category'] != 'Cant rent']

# Drop rows with 'Maintenance' in the 'Category' column which was the Maintenance category
df = df[df['Category'] != 'Maintaince']


In [ ]:
df['Category'].value_counts()

### Cleaning the Product Number column
- [X] Clean junk that isn't a number
- [X] Replace repair tag numbers to 0
- [X] Create a unique number
    - Set up sample for analysis later in specific bikes
    - template: Location/product/product_num
        - example: WaterfrontDouble_surrey5 or WFDS05


In [ ]:
print('Null Values')
# Fill null values in the 'Product_Num' column with 0
df['Product_Num'] = df['Product_Num'].fillna(0)
df['Product_Num'].isnull().sum()

In [ ]:
print('Each location had product numbers. Example: (Chopper Edinger #3 was unique to location and product included)')

# Remove all non-digit characters and trailing zeros from the 'Product_Num' column
df['Product_Num'] = df['Product_Num'].astype(str).replace('[^\d]+|(?<=\d)0+(?=\d)', '', regex=True).str.rstrip('0')
df['Product_Num'].value_counts()


In [ ]:
print("Numbered_products will be used as a dataset we can do analysis on")
# Select rows where the 'Product_Num' column is not equal to '0'
numbered_products = df[df['Product_Num'] != '0']
fig, ax = plt.subplots(figsize=(20,8))
numbered_products['Product'].hist(ax=ax)
# Set axis labels and plot title
ax.set_xlabel('Year-Month')
ax.set_ylabel('Number of Reports')
ax.set_title('Histogram of the reports that has product numbers')
print("We can use these bikes to see how bike maintaince can behaves over time and dependent on Location")

In [ ]:
print('Error in product number, Product number at warner was repair tag id number, so Warner tags and potentially Edinger are inaccurate')
# Convert 'Product_Num' column to numeric type
df['Product_Num'] = pd.to_numeric(df['Product_Num'], errors='coerce')

# Filter for rows with 'Product_Num' greater than 30
high_product_num = df[df['Product_Num'] > 30]

# Print information about high product numbers
for index, row in high_product_num.iterrows():
    print(f"Product {row['Product']} from {row['Location']} has a Product_Num of {row['Product_Num']}.")

# Filter for rows with NaN values in the 'Product_Num' column
nan_product_num = df[df['Product_Num'].isna()]

# Define a lambda function to combine columns and handle missing values
combine_columns = lambda row: f"{row['Location'].replace('Waterfront', 'W.A.').replace('Yorba Linda', 'Y.L.').replace('Edinger', 'M.E.').replace('Warner', 'M.W.')}_{row['Product']}_{row['Product_Num']}" if pd.notnull(row['Product_Num']) else ""

# Apply the lambda function to create a new column 'Location_Product_Num'
df['Location_Product_Num'] = df.apply(combine_columns, axis=1)

In [ ]:
# Set Product_Num to 0 for repair tags at warner
df.loc[(df['Location'].str.contains('Warner')) & (df['Product_Num'] > 700), 'Product_Num'] = 0
# drop row where mulitple cruisers were worked since it was supposed to be maintaince and not down bikes
df.drop(df[(df['Location'].str.contains('Waterfront')) & (df['Product_Num'] > 20000)].index, inplace=True)
# Display the high numbers
high_product_num = df[df['Product_Num'] > 30]

for index, row in high_product_num.iterrows():
    print(f"Product {row['Product']} from {row['Location']} has a Product_Num of {row['Product_Num']}.")
# the five outputs all have high number product numbers due to 
#transfers and based on the serial number  

In [ ]:
df['Product_Num'].value_counts()

### Create a YearMonth column

In [ ]:
# Create a new column with the year and month extracted from the 'Date' column
df['YearMonth'] = df['Down_Date'].dt.strftime('%Y-%m')

# Group the data by year and month, and count the number of reports in each group
monthly_count = df.groupby('YearMonth')['Down_Date'].count()

# Plot the resulting data as a line graph
plt.figure(figsize=(13, 8))
plt.plot(monthly_count.index, monthly_count.values)

plt.xlabel('Year-Month')
plt.ylabel('Number of Reports')
plt.title('Monthly Report Count')
plt.show()


## Create a Fixed Days Columns

In [ ]:
# Create a new column called 'fixed_days'
df['Fixed_Days'] = (df['Fixed_Date'] - df['Down_Date']).dt.days
df['Fixed_Days'].hist()
plt.title('Count of Fixed Days Historgram')

In [ ]:
# Identify Outliers
# Create a boolean mask to identify rows where Fixed_Days is over 35
mask = df['Fixed_Days'] > 80

# Use the mask to filter the original DataFrame and create a new variable
fixed_days_over_80 = df[mask]

# Display the new variable
fixed_days_over_80['Fixed_Days'].hist()
plt.title('Identify Outliers over 80 days')

In [ ]:
fixed_days_over_80['Fixed_Days'].describe()

In [ ]:
fixed_days_85 = df[df['Fixed_Days'] == 85]
fixed_days_85.head()

In [ ]:
dec_30_reports = df[df['Fixed_Date']== '2020-12-30']
dec_30_reports.head()

In [ ]:
# Use the drop() method to drop the selected rows
df = df.drop(dec_30_reports.index)
# Verify that the rows have been dropped
df['Fixed_Days'].hist()

In [ ]:
df['Fixed_Days'].hist()

In [ ]:
print('Outliers of more than 80 days being down')
outliers = df[df['Fixed_Days'] > 80]
outliers.describe()

## Ordered Variable Dataframe

In [ ]:
# Create a new DataFrame where the 'Status' column contains 'Order'
ordered_df = df[df['Status'].str.contains('Order') | df['Status'].str.contains('Received')]
ordered_df['Fixed_Days'].hist()
plt.title('Reports that needed parts to be ordered or recieved with fixed days')

# Outliers

In [ ]:
# Define a function to investigate outliers
def investigate_outliers(df, column):
    # Create a boxplot to visualize the distribution and outliers
    fig, ax = plt.subplots(figsize=(8,6))
    ax.boxplot(df['Fixed_Days'])
    ax.set_title(f"Distribution and outliers in {column}")
    ax.set_ylabel(column)

    # Calculate the upper and lower bounds for outliers
    q1 = df['Fixed_Days'].quantile(0.25)
    q3 = df['Fixed_Days'].quantile(0.75)
    iqr = q3 - q1
    upper_bound = q3 + 1.5 * iqr
    lower_bound = q1 - 1.5 * iqr
    
    # Count the number of outliers
    outliers = df[(df['Fixed_Days'] < lower_bound) | (df['Fixed_Days'] > upper_bound)]
    num_outliers = len(outliers)
    
    # Print the number of outliers and their details
    print(f"\nThere are {num_outliers} outliers in the {column} column:")

# Call the function to investigate outliers in the 'Days' column
investigate_outliers(df, 'Fixed_Days')


## Medians and Averages by Locations

In [ ]:
# Calculate average and median values by location and day
grouped = df.groupby(['Location', 'Down_Date']).mean().reset_index()
average_by_location = grouped.groupby('Location')['Fixed_Days'].mean()
median_by_location = grouped.groupby('Location')['Fixed_Days'].median()
for location in df['Location'].unique():
    print(f"Location: {location}")
    print(f"Average fixed days: {average_by_location[location]}")
    print(f"Median fixed days: {median_by_location[location]}")
    print()

# What was the busiest time of the year

In [ ]:
# Count the frequency of each Down_Date value and sort in descending order
date_counts = df['Down_Date'].value_counts().sort_values(ascending=False)
# Get the date with the highest count
busiest_date = date_counts.index[0]
# Count the number of reports for the busiest date
busiest_date_count = date_counts[0]
print(f"The busiest time of the year is {busiest_date} with {busiest_date_count} reports.")


## Description Word Plot

In [ ]:
from wordcloud import WordCloud
all_words = ''
for arg in df["Description"]: 

    tokens = arg.split()  
      
    all_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 700, height = 700, 
                background_color ='white', 
                min_font_size = 10).generate(all_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (13, 13), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

In [ ]:
# count the number of occurrences of each unique value in the Description column
top_descriptions = df['Description'].value_counts()

# print the top 10 most frequent descriptions and their counts
print("Top Description Results:")
print(top_descriptions.head(15))

## Tire Related Problems Overtime

In [ ]:
# Create a new column that contains a boolean value indicating whether the description contains the word "tire"
df["Has_Tire"] = df["Description"].str.contains("tire")

# Group the data by Down_Date and count the number of rows for each date that have a description containing the word "tire"
tire_counts = df.groupby("YearMonth")["Has_Tire"].sum()
# Create a histogram of the tire counts
fig, ax = plt.subplots(figsize=(15, 6))
ax.hist(df[df["Has_Tire"]]["Down_Date"], bins=15)
ax.set_title("Distribution of Maintenance Reports Containing 'Tire'")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Reports")
plt.show()

In [ ]:
print(tire_counts)

## Chain Related Problems Overtime

In [ ]:
# Create a new column that contains a boolean value indicating whether the description contains the word "tire"
df["Has_Chain"] = df["Description"].str.contains("chain")

# Group the data by Down_Date and count the number of rows for each date that have a description containing the word "tire"
chain_counts = df.groupby("YearMonth")["Has_Chain"].sum()
# Create a histogram of the tire counts
fig, ax = plt.subplots(figsize=(15, 6))
ax.hist(df[df["Has_Chain"]]["Down_Date"], bins=8)
ax.set_title("Distribution of Maintenance Reports Containing 'Chain'")
ax.set_xlabel("Date")
ax.set_ylabel("Number of Reports")
plt.show()

In [ ]:

# Create a new column that contains a boolean value indicating whether the description contains the word "brake"
df["Has_Brake"] = df["Description"].str.contains("brake")

# Group the data by YearMonth and count the number of rows for each month that have a description containing the word "brake"
brake_counts = df.groupby("YearMonth")["Has_Brake"].sum()

# Create a histogram of the brake counts
fig, ax = plt.subplots(figsize=(15,6))
ax.hist(df[df["Has_Brake"]]["Down_Date"], bins=9)
ax.set_title("Distribution of Maintenance Reports Containing 'Brake'")
ax.set_xlabel("Number of Reports")
ax.set_ylabel("Frequency")
plt.show()


In [ ]:

# Group the data by date and count the number of maintenance reports for each variable
df_count = df.groupby(["Down_Date"])[["Has_Tire", "Has_Chain"]].sum()

# Filter out any days with zero reports
df_count = df_count[(df_count.T != 0).any()]

# Create a scatter plot of the count over time for both variables
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(df_count.index, df_count["Has_Tire"], color='blue', alpha=0.5, label="Has Tire")
ax.scatter(df_count.index, df_count["Has_Chain"], color='green', alpha=0.5, label="Has Chain")
ax.set_title("Count of Maintenance Reports with 'Tire' and 'Chain'")
ax.set_xlabel("Date")
ax.set_ylabel("Count of Maintenance Reports")
ax.legend()
plt.show()

# How much did ordering have an effect on the days to fix the products?

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
ax.hist(ordered_df['Fixed_Days'], bins=15)
ax.set_title("Ordered Parts")
ax.set_xlabel("Days")
ax.set_ylabel("Frequency")

In [ ]:
# Calculate the median days to fix for the entire dataset and the ordered bikes
median_all_bikes = df['Fixed_Date'].median()
median_ordered_bikes = ordered_df['Fixed_Date'].median()

# Create a histogram of days to fix for all bikes and ordered bikes
plt.figure(figsize=(10,8))
plt.hist(df['Fixed_Date'], alpha=0.5, label='All Bikes')
plt.hist(ordered_df['Fixed_Date'], alpha=0.5, label='Ordered Bikes')
plt.axvline(median_all_bikes, color='blue', linestyle='dashed', linewidth=1, label=f'Median All Bikes: {median_all_bikes:.1f}')
plt.axvline(median_ordered_bikes, color='orange', linestyle='dashed', linewidth=1, label=f'Median Ordered Bikes: {median_ordered_bikes:.1f}')
plt.xlabel('Days to Fix')
plt.ylabel('Frequency')
plt.legend()
plt.show()


In [ ]:
# Create a histogram to show the frequency of values
fig, ax = plt.subplots(figsize=(10,8))
ax.hist(df['Fixed_Days'], bins=25)
ax.set_title("Histogram of all Products ")
ax.set_xlabel("Days")
ax.set_ylabel("Frequency")

# **Productivity**

In [ ]:
# group the data by month and calculate the average Days_to_Fix for each month
monthly_avg = df.groupby(pd.Grouper(key='Down_Date', freq='M'))['Fixed_Days'].mean()

# plot the time series
plt.figure(figsize=(13, 8))
plt.plot(monthly_avg)
plt.xlabel('Month')
plt.ylabel('Average Days to Fix')
plt.title('Time Series of Average Days to Fix')
plt.show()

In [ ]:
df = df.sort_values('Down_Date')
# Calculate the moving average
window_size = 7  
# Adjust the window size as needed
df['MovingAverage'] = df['Fixed_Days'].rolling(window=window_size).mean()

# Create the scatter plot
fig, ax = plt.subplots(figsize=(20, 10))
ax.scatter(df['Down_Date'], df['Fixed_Days'], label='Fixed Days', alpha=0.5)

# Plot the moving average line
ax.plot(df['Down_Date'], df['MovingAverage'], color='red', label=f'{window_size}-Day Moving Average')

# Set labels and title
ax.set_xlabel('Dates')
ax.set_ylabel('Days the Product was Down')
ax.set_title('Mechanic Productivity')

# Add a legend
ax.legend()

# Show the plot
plt.show()

## Mechanic Notes Word Cloud

In [ ]:
from wordcloud import WordCloud
# Get all words after "need" in the Mechanic_Notes column
need_words = ' '.join(df['Mechanic_Notes'].str.split('needs').str[1].dropna())
# Generate word cloud
wordcloud = WordCloud(width=800, height=800, background_color='white', colormap='Blues').generate(need_words)

# Display word cloud
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

# Fleet KPI


## Waterfront Fleet

In [ ]:
import seaborn as sns

# Filter the data for the Waterfront location
waterfront_data = df.loc[df['Location'] == 'Waterfront', :]

# Create a boxplot of the quantity of products sold
sns.boxplot(data=waterfront_data, y='Fixed_Days')

# Set plot title and axis labels
plt.title('Distribution of Product at Waterfront Location')
plt.xlabel('Waterfront')
plt.ylabel('Products')

# Display the plot
plt.show()


# Mean Time to Repair KPI

In [ ]:
# Calculate the MTTR
mttr = df['Fixed_Days'].mean()

print(f"The Mean Time To Repair (MTTR) is {mttr:.2f} days.")


In [ ]:
# Group the data by date and calculate the MTTR for each date
mttr_by_date = df.groupby('Fixed_Date')['Fixed_Days'].mean()

# Create a time series plot of the MTTR over time
plt.figure(figsize=(20,6))
plt.plot(mttr_by_date.index, mttr_by_date.values)
plt.xlabel('Date')
plt.ylabel('Mean Time To Repair (MTTR)')
plt.title('MTTR over Time')
plt.show()

# Repair Success Rate

In [ ]:
# Group the data by unique bike and count the number of repairs
num_repairs = df.groupby('Location_Product_Num').size()

# Filter the data to only include bikes that have been repaired more than 2 times
filtered_data = df[df['Location_Product_Num'].isin(num_repairs[num_repairs > 2].index)]

# Sort the data by bike and repair date
filtered_data.sort_values(['Location_Product_Num', 'Down_Date'], inplace=True)

# Calculate the time between each repair for each bike
filtered_data['Time_To_Next_Break'] = filtered_data.groupby('Location_Product_Num')['Down_Date'].diff()

# Convert the time difference to days 
filtered_data['Time_To_Next_Break'] = filtered_data['Time_To_Next_Break'].dt.days

# Drop the first row for each bike (since it doesn't have a previous repair date)
filtered_data = filtered_data.groupby('Location_Product_Num').apply(lambda x: x.iloc[1:])

# Plot the distribution of time between repairs
plt.hist(filtered_data['Time_To_Next_Break'], bins=30)
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.title('Distribution of Time Between Repairs (Bikes Repaired > 2 Times)')
plt.show()


# The Most Prone Bike to Break 

In [ ]:
# Group the data by unique bike and count the number of repairs
num_repairs = df.groupby('Location_Product_Num').size()

# Sort the result in descending order
most_repaired_bike = df['Location_Product_Num'].value_counts().index[1]
print(f"The bike with the most repairs is {most_repaired_bike}")
